In [43]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import polars as pl
import requests
import json
import gzip
import json
import os
from datetime import datetime

In [2]:
predictions_url = 'https://api-v3.mbta.com/prediction'
with open('mbta_api_key.txt') as file:
    api_key = file.read()

In [8]:
routes = requests.get('https://api-v3.mbta.com/routes').json()
for route in routes['data']:
    route_id = route['id']
    print(f'Route ID: {route_id}')

Route ID: Red
Route ID: Mattapan
Route ID: Orange
Route ID: Green-B
Route ID: Green-C
Route ID: Green-D
Route ID: Green-E
Route ID: Blue
Route ID: 741
Route ID: 742
Route ID: 743
Route ID: 751
Route ID: 749
Route ID: 746
Route ID: CR-Fairmount
Route ID: CR-Fitchburg
Route ID: CR-Worcester
Route ID: CR-Franklin
Route ID: CR-Greenbush
Route ID: CR-Haverhill
Route ID: CR-Kingston
Route ID: CR-Lowell
Route ID: CR-Middleborough
Route ID: CR-Needham
Route ID: CR-Newburyport
Route ID: CR-Providence
Route ID: CR-Foxboro
Route ID: Boat-F4
Route ID: Boat-F1
Route ID: Boat-EastBoston
Route ID: Boat-Lynn
Route ID: Boat-F6
Route ID: 747
Route ID: 708
Route ID: 1
Route ID: 4
Route ID: 7
Route ID: 8
Route ID: 9
Route ID: 10
Route ID: 11
Route ID: 14
Route ID: 15
Route ID: 16
Route ID: 17
Route ID: 18
Route ID: 19
Route ID: 21
Route ID: 22
Route ID: 23
Route ID: 24
Route ID: 26
Route ID: 28
Route ID: 29
Route ID: 30
Route ID: 31
Route ID: 32
Route ID: 33
Route ID: 34
Route ID: 34E
Route ID: 35
Route I

In [4]:
stops = requests.get('https://api-v3.mbta.com/stops', params={'filter[route]': 'Green-D'}).json()
for stop in stops['data']:
    stop_id = stop['id']
    stop_name = stop['attributes']['name']
    print(f'Stop ID: {stop_id}, Stop Name: {stop_name}')

Stop ID: place-river, Stop Name: Riverside
Stop ID: place-woodl, Stop Name: Woodland
Stop ID: place-waban, Stop Name: Waban
Stop ID: place-eliot, Stop Name: Eliot
Stop ID: place-newtn, Stop Name: Newton Highlands
Stop ID: place-newto, Stop Name: Newton Centre
Stop ID: place-chhil, Stop Name: Chestnut Hill
Stop ID: place-rsmnl, Stop Name: Reservoir
Stop ID: place-bcnfd, Stop Name: Beaconsfield
Stop ID: place-brkhl, Stop Name: Brookline Hills
Stop ID: place-bvmnl, Stop Name: Brookline Village
Stop ID: place-longw, Stop Name: Longwood
Stop ID: place-fenwy, Stop Name: Fenway
Stop ID: place-kencl, Stop Name: Kenmore
Stop ID: place-hymnl, Stop Name: Hynes Convention Center
Stop ID: place-coecl, Stop Name: Copley
Stop ID: place-armnl, Stop Name: Arlington
Stop ID: place-boyls, Stop Name: Boylston
Stop ID: place-pktrm, Stop Name: Park Street
Stop ID: place-gover, Stop Name: Government Center
Stop ID: place-haecl, Stop Name: Haymarket
Stop ID: place-north, Stop Name: North Station
Stop ID: plac

In [5]:
params1 = {'api_key': api_key,
          'filter[stop]': 'place-gover'}

In [6]:
response1 = requests.get(f'https://api-v3.mbta.com/predictions?filter[stop]=place-gover&api_key={api_key}').json()

In [7]:
response1['data'][0]

{'attributes': {'arrival_time': None,
  'arrival_uncertainty': None,
  'departure_time': '2024-05-23T16:20:16-04:00',
  'departure_uncertainty': 360,
  'direction_id': 0,
  'last_trip': False,
  'revenue': 'REVENUE',
  'schedule_relationship': 'ADDED',
  'status': None,
  'stop_sequence': 40},
 'id': 'prediction-ADDED-1581915212-70202-40',
 'relationships': {'route': {'data': {'id': 'Green-B', 'type': 'route'}},
  'stop': {'data': {'id': '70202', 'type': 'stop'}},
  'trip': {'data': {'id': 'ADDED-1581915212', 'type': 'trip'}},
  'vehicle': {'data': {'id': 'G-10319', 'type': 'vehicle'}}},
 'type': 'prediction'}

In [28]:
test_predictions_path = 'data/api_fetches/predictions_20240523160239.json'
test_preds = pd.read_json(test_predictions_path)

In [29]:
test_preds.head()

,attributes,id,relationships,type
0,"{'arrival_time': None, 'arrival_uncertainty': ...",prediction-ADDED-1581915229-70206-20,"{'route': {'data': {'id': 'Green-D', 'type': '...",prediction
1,"{'arrival_time': None, 'arrival_uncertainty': ...",prediction-CR-660226-115-BNT-0000-02-0,"{'route': {'data': {'id': 'CR-Newburyport', 't...",prediction
2,"{'arrival_time': '2024-05-23T15:58:00-04:00', ...",prediction-61804902-70205-630,"{'route': {'data': {'id': 'Green-D', 'type': '...",prediction
3,"{'arrival_time': '2024-05-23T15:58:32-04:00', ...",prediction-61805113-70206-20,"{'route': {'data': {'id': 'Green-D', 'type': '...",prediction
4,"{'arrival_time': '2024-05-23T16:00:43-04:00', ...",prediction-63029051-70027-130,"{'route': {'data': {'id': 'Orange', 'type': 'r...",prediction


In [30]:
test_preds['attributes'].iloc[0]

{'arrival_time': None,
 'arrival_uncertainty': None,
 'departure_time': '2024-05-23T16:02:36-04:00',
 'departure_uncertainty': 360,
 'direction_id': 0,
 'last_trip': False,
 'revenue': 'REVENUE',
 'schedule_relationship': 'ADDED',
 'status': None,
 'stop_sequence': 20}

In [31]:
test_preds['relationships'].iloc[0]

{'route': {'data': {'id': 'Green-D', 'type': 'route'}},
 'stop': {'data': {'id': '70206', 'type': 'stop'}},
 'trip': {'data': {'id': 'ADDED-1581915229', 'type': 'trip'}},
 'vehicle': {'data': {'id': 'G-10286', 'type': 'vehicle'}}}

In [32]:
def extract_ids(relationship):
    return {
        'route_id': relationship.get('route', {}).get('data', {}).get('id', None),
        'stop_id': relationship.get('stop', {}).get('data', {}).get('id', None),
        'trip_id': relationship.get('trip', {}).get('data', {}).get('id', None),
        'vehicle_id': relationship.get('vehicle', {}).get('data', {}).get('id', None)
    }

In [33]:
ids_df = test_preds['relationships'].apply(extract_ids).apply(pd.Series)
test_preds = pd.concat([test_preds, ids_df], axis=1)
test_preds = test_preds.drop(columns=['relationships'])

In [34]:
test_preds

,attributes,id,type,route_id,stop_id,trip_id
0,"{'arrival_time': None, 'arrival_uncertainty': ...",prediction-ADDED-1581915229-70206-20,prediction,Green-D,70206,ADDED-1581915229
1,"{'arrival_time': None, 'arrival_uncertainty': ...",prediction-CR-660226-115-BNT-0000-02-0,prediction,CR-Newburyport,BNT-0000-02,CR-660226-115
2,"{'arrival_time': '2024-05-23T15:58:00-04:00', ...",prediction-61804902-70205-630,prediction,Green-D,70205,61804902
3,"{'arrival_time': '2024-05-23T15:58:32-04:00', ...",prediction-61805113-70206-20,prediction,Green-D,70206,61805113
4,"{'arrival_time': '2024-05-23T16:00:43-04:00', ...",prediction-63029051-70027-130,prediction,Orange,70027,63029051
...,...,...,...,...,...,...
134,"{'arrival_time': '2024-05-23T17:20:36-04:00', ...",prediction-ADDED-1581915207-70079-90,prediction,Red,70079,ADDED-1581915207
135,"{'arrival_time': '2024-05-23T17:22:32-04:00', ...",prediction-61946751-74617-10,prediction,743,74617,61946751
136,"{'arrival_time': '2024-05-23T17:24:53-04:00', ...",prediction-61946473-74617-10,prediction,741,74617,61946473
137,"{'arrival_time': '2024-05-23T17:27:22-04:00', ...",prediction-61946465-74617-10,prediction,741,74617,61946465


In [44]:
with open('data/api_fetches/predictions_20240523160239.json', 'r') as file:
    json_data = json.load(file)
test_preds = pd.json_normalize(json_data)

In [46]:
test_preds.columns

Index(['id', 'type', 'attributes.arrival_time',
       'attributes.arrival_uncertainty', 'attributes.departure_time',
       'attributes.departure_uncertainty', 'attributes.direction_id',
       'attributes.last_trip', 'attributes.revenue',
       'attributes.schedule_relationship', 'attributes.status',
       'attributes.stop_sequence', 'relationships.route.data.id',
       'relationships.route.data.type', 'relationships.stop.data.id',
       'relationships.stop.data.type', 'relationships.trip.data.id',
       'relationships.trip.data.type', 'relationships.vehicle.data.id',
       'relationships.vehicle.data.type', 'relationships.vehicle.data'],
      dtype='object')

In [47]:
test_preds.head()

,id,type,attributes.arrival_time,attributes.arrival_uncertainty,attributes.departure_time,attributes.departure_uncertainty,attributes.direction_id,attributes.last_trip,attributes.revenue,attributes.schedule_relationship,...,attributes.stop_sequence,relationships.route.data.id,relationships.route.data.type,relationships.stop.data.id,relationships.stop.data.type,relationships.trip.data.id,relationships.trip.data.type,relationships.vehicle.data.id,relationships.vehicle.data.type,relationships.vehicle.data
0,prediction-ADDED-1581915229-70206-20,prediction,None,NaN,2024-05-23T16:02:36-04:00,360.0,0,False,REVENUE,ADDED,...,20,Green-D,route,70206,stop,ADDED-1581915229,trip,G-10286,vehicle,NaN
1,prediction-CR-660226-115-BNT-0000-02-0,prediction,None,NaN,2024-05-23T16:05:00-04:00,NaN,0,False,REVENUE,None,...,0,CR-Newburyport,route,BNT-0000-02,stop,CR-660226-115,trip,NaN,NaN,NaN
2,prediction-61804902-70205-630,prediction,2024-05-23T15:58:00-04:00,60.0,2024-05-23T15:58:53-04:00,60.0,1,False,REVENUE,None,...,630,Green-D,route,70205,stop,61804902,trip,G-10274,vehicle,NaN
3,prediction-61805113-70206-20,prediction,2024-05-23T15:58:32-04:00,60.0,2024-05-23T15:59:21-04:00,60.0,0,False,REVENUE,None,...,20,Green-D,route,70206,stop,61805113,trip,G-10001,vehicle,NaN
4,prediction-63029051-70027-130,prediction,2024-05-23T16:00:43-04:00,60.0,2024-05-23T16:01:35-04:00,60.0,1,False,REVENUE,None,...,130,Orange,route,70027,stop,63029051,trip,O-547C7297,vehicle,NaN


In [58]:
len(requests.get('https://api-v3.mbta.com/vehicles', params={'filter[route_type]': 0}).json()['data'])

62

In [59]:
len(requests.get('https://api-v3.mbta.com/vehicles', params={'filter[route_type]': 1}).json()['data'])

45

In [60]:
len(requests.get('https://api-v3.mbta.com/vehicles', params={'filter[route_type]': 2}).json()['data'])

37

In [61]:
len(requests.get('https://api-v3.mbta.com/vehicles', params={'filter[route_type]': 3}).json()['data'])

521

In [63]:
len(requests.get('https://api-v3.mbta.com/vehicles', params={'filter[route_type]': '[0, 1, 2, 3]'}).json()['data'])

664

In [86]:
stops = requests.get('https://api-v3.mbta.com/stops', params={'filter[route_type]': str([0, 1, 2, 3])}).json()
for stop in stops['data']:
    stop_id = stop['id']
    stop_name = stop['attributes']['name']
    print(f'Stop ID: {stop_id}, Stop Name: {stop_name}')

Stop ID: 24486, Stop Name: Broadway @ Hampshire St
Stop ID: 8556, Stop Name: Spiers Rd @ Dedham St
Stop ID: door-pktrm-exitashbrain, Stop Name: Park Street - Escalator to Tremont St (South)
Stop ID: node-dwnxg-112fare-paid, Stop Name: Downtown Crossing
Stop ID: 3124, Stop Name: W Squantum St @ Farrington St
Stop ID: 32004, Stop Name: Quincy Center
Stop ID: 1616, Stop Name: Margaretta Dr @ Crown Point Dr
Stop ID: node-sstat-north-farepaid, Stop Name: South Station
Stop ID: 4554, Stop Name: Opp 30 Highland Ave
Stop ID: 38481, Stop Name: Franklin St opp Trescott St
Stop ID: 2406, Stop Name: College Ave @ Dearborn Rd
Stop ID: 2128, Stop Name: Trapelo Rd opp Poplar St
Stop ID: 6230, Stop Name: Central St @ Jasper St
Stop ID: 1268, Stop Name: Washington St @ Shepard St
Stop ID: node-149-lobby, Stop Name: Tufts Medical Center
Stop ID: 5250, Stop Name: 1437 Centre St opp Ardale St
Stop ID: 171, Stop Name: Berkeley St @ Columbus Ave
Stop ID: 16132, Stop Name: Essex St opp Burrill St
Stop ID: 14

In [90]:
pd.json_normalize(requests.get('https://api-v3.mbta.com/predictions', params={'filter[route]': 'Green-D'}).json()['data'])

,id,type,attributes.arrival_time,attributes.arrival_uncertainty,attributes.departure_time,attributes.departure_uncertainty,attributes.direction_id,attributes.last_trip,attributes.revenue,attributes.schedule_relationship,attributes.status,attributes.stop_sequence,relationships.route.data.id,relationships.route.data.type,relationships.stop.data.id,relationships.stop.data.type,relationships.trip.data.id,relationships.trip.data.type,relationships.vehicle.data.id,relationships.vehicle.data.type
0,prediction-61805051-70160-310,prediction,None,NaN,2024-05-23T19:11:22-04:00,60.0,1,False,REVENUE,None,None,310,Green-D,route,70160,stop,61805051,trip,G-10380,vehicle
1,prediction-61805008-70176-390,prediction,2024-05-23T19:03:56-04:00,60.0,2024-05-23T19:04:51-04:00,60.0,1,False,REVENUE,None,None,390,Green-D,route,70176,stop,61805008,trip,G-10115,vehicle
2,prediction-61805095-70159-90,prediction,2024-05-23T19:04:08-04:00,60.0,2024-05-23T19:04:43-04:00,60.0,0,False,REVENUE,None,None,90,Green-D,route,70159,stop,61805095,trip,G-10274,vehicle
3,prediction-61805008-70178-400,prediction,2024-05-23T19:06:21-04:00,60.0,2024-05-23T19:06:57-04:00,60.0,1,False,REVENUE,None,None,400,Green-D,route,70178,stop,61805008,trip,G-10115,vehicle
4,prediction-61805095-70157-100,prediction,2024-05-23T19:06:22-04:00,60.0,2024-05-23T19:07:03-04:00,60.0,0,False,REVENUE,None,None,100,Green-D,route,70157,stop,61805095,trip,G-10274,vehicle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,prediction-61804908-70161-570,prediction,2024-05-23T20:03:54-04:00,60.0,None,NaN,0,False,REVENUE,None,None,570,Green-D,route,70161,stop,61804908,trip,G-10088,vehicle
219,prediction-61805037-70503-660,prediction,2024-05-23T20:03:56-04:00,60.0,None,NaN,1,False,REVENUE,None,None,660,Green-D,route,70503,stop,61805037,trip,G-10367,vehicle
220,prediction-61805051-70207-640,prediction,2024-05-23T20:04:42-04:00,60.0,2024-05-23T20:05:19-04:00,60.0,1,False,REVENUE,None,None,640,Green-D,route,70207,stop,61805051,trip,G-10380,vehicle
221,prediction-61805051-70501-650,prediction,2024-05-23T20:06:16-04:00,60.0,2024-05-23T20:07:10-04:00,60.0,1,False,REVENUE,None,None,650,Green-D,route,70501,stop,61805051,trip,G-10380,vehicle
